In [1]:
# Okay, I must not be the only one who attepted to replicate some medium or towards data science
# we should probably rename it or create a new one toward AGI guides on LLM and become 
# incessantly frustrated about how to properly setup all the libary versions and drivers.
# Or maybe not, probably not because it is not that complicated. Alas, besides going over some startup LLM
# pondering some idea projects and applications such as what Mistral actualyl good for, i'll also include some
# sample code that automatically captures your python transformers, langchain, openai lbirary versions and saves them for
# later use. If you need it.


In [8]:
# !conda install bitsandbytes accelerate xformers einops langchain faiss-cpu transformers sentence-transformers
import torch
# import pytorch
torch.zeros(1).cuda()

# The magic line to fix torch due to cuda version compatibility issues
  # 378  conda install pytorch=*=*cuda* cudatoolkit -c pytorch

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Device: cuda


In [4]:
from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

# >>> Device: cuda
# >>> Tesla T4
import pkg_resources, sys, os


Device: cuda
NVIDIA GeForce RTX 4080 Laptop GPU


In [5]:
# echo and get the current conda env name
!echo $CONDA_DEFAULT_ENV

conda_env_name = os.getenv('CONDA_DEFAULT_ENV')

mistral_7B_mlm


In [6]:
# Perform a CUDA device availability check
def get_cuda_device():
    if torch.cuda.is_available:
        return "cuda:0"
    else:
        return "cpu"

def get_cuda_device():
    if torch.cuda.is_available:
        return "cuda:0"
    else:
        return "cpu"

In [7]:
orig_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded" # quantized model
bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                # load_in_8bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                )

tokenizer = AutoTokenizer.from_pretrained(orig_model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, 
                                             quantization_config=bnb_config, 
                                             device_map="auto")

# text = "What is Mistral? Write a short answer but apt answer that would pass a college test."
# mistral_llm.invoke(text)
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    temperature=1.0,
    return_full_text=True,
    max_new_tokens=378,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print(model)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


### Creating the LLM Pipeline with what appears to be Huggingface pipeline
- transformers.pipeline
- Now let's create the LLM pipeline. It is very simple to do using the transformers pipeline capability.

#### Using Langchain framework with LLMs
- LangChain is a cross platform python framework that supports diferent language models without the ai engineer having to write additional support code.
- 

In [8]:
device = "cuda:0"
messages = [
        {"role": "user", "content": "What is the usual composition of rocket fuel?"}
]

chat_encodes = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = chat_encodes.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is the usual composition of rocket fuel? [/INST] The usual composition of rocket fuel varies depending on the specific type of rocket and its intended purpose. However, common ingredients in rocket fuel include nitrogen oxides (such as nitrogen peroxide or monomethyl hydrazine), alcohols (such as ethylene oxide or isopropyl alcohol), ketones (such as acetone or benzene), and other chemicals that produce hot gases when burned. These chemicals are chosen for their ability to produce high levels of thrust, efficient combustion, and low toxicity. The precise mixture of fuels used in a rocket can be optimized to achieve the desired performance characteristics.</s>


In [9]:
import textwrap
def get_llm(text_prompt):
    return mistral_llm.invoke(text_prompt)

# Define the column width for text wrapping (Medium's typical width is around 80 characters)
COLUMN_WIDTH = 76

def clean_text(text):
    # Remove newline characters and any other special characters you want to strip
    cleaned_text = text.replace('\n', ' ').replace('\r', '').strip()
    return cleaned_text

def wrap_text(text, width):
    # Use textwrap to wrap text to the specified width
    wrapped_text = textwrap.fill(text, width=width)
    return wrapped_text

def wrap_text_with_comments(text, width):
    # Wrap the text to the specified width
    wrapped_text = textwrap.fill(text, width=width)
    # Split the wrapped text into lines
    lines = wrapped_text.split('\n')
    # Add a comment character at the start of each line
    commented_lines = ['# ' + line for line in lines]
    # Join the lines back together
    commented_text = '\n'.join(commented_lines)
    return commented_text


def get_llm_response(text_prompt):
    # Get the response from the model
    response = mistral_llm.invoke(text_prompt)
    # Clean the output text
    cleaned_response = clean_text(response)
    # Wrap the cleaned response text to the specified column width
    wrapped_response = wrap_text_with_comments(cleaned_response, COLUMN_WIDTH)
    return wrapped_response

In [10]:
text = "What is the usual composition of rocket fuel?"
mistral_llm.invoke(text)
text = "What is Mistral? Write a short answer but apt answer that would pass a college test."
mistral_llm.invoke(text)

'What is Mistral? Write a short answer but apt answer that would pass a college test.\n\nMistral is a type of wind that blows from the Mediterranean Sea towards the French Alps. It is known for its strong and cold winds, which can reach speeds of up to 100 km/h (62 mph). The mistral is a common weather phenomenon in the region and is often associated with extreme conditions, such as heavy snowfall and freezing temperatures.'

In [11]:
mistral_out = mistral_llm.invoke(text)
print(mistral_out)

What is Mistral? Write a short answer but apt answer that would pass a college test.

Mistral is a type of wind that blows from the Mediterranean Sea towards the French Alps. It is known for its strong and cold winds, which can reach speeds of up to 100 km/h (62 mph). The mistral is a common weather phenomenon in the region and is often associated with extreme conditions, such as heavy snowfall and freezing temperatures.


In [12]:
clean_response = get_llm_response(text)
print(clean_response)

# What is Mistral? Write a short answer but apt answer that would pass a
# college test.  Mistral is a type of wind that blows from the Mediterranean
# Sea towards the French Alps. It is known for its strong and cold winds,
# which can reach speeds of up to 100 km/h (62 mph). The mistral is a common
# weather phenomenon in the region and is often associated with extreme
# conditions, such as heavy snowfall and freezing temperatures.


In [13]:
# quick_test = "What is Mistral AI? Please write a cohesive, apt answer that would be deemed college exam worthy."
# mistral_llm.invoke(quick_test)

In [14]:
quick_test = "What is Mistral AI? Please write a cohesive, apt answer that would be deemed college exam worthy."
# mistral_llm.invoke(quick_test)
print(get_llm_response(quick_test))

# What is Mistral AI? Please write a cohesive, apt answer that would be deemed
# college exam worthy.  Mistral AI is an innovative company based in Paris,
# France, developing large language models. These models are capable of
# generating human-like text and can be fine-tuned for specific tasks such as
# translation, summarization, and question answering. The company's flagship
# product, Mistral, is a powerful language model that has been trained on vast
# amounts of data from the internet and can generate high-quality text in over
# 100 languages.  One of the key features of Mistral AI is its ability to
# generate text that is both fluent and contextually relevant. This is
# achieved through the use of advanced natural language processing techniques
# and machine learning algorithms that allow the model to understand the
# meaning and context of words and phrases. Additionally, Mistral AI's models
# are designed to be highly customizable, allowing users to fine-tune them for
# speci

#### Prompt input model using Transformers pipes
        

In [15]:
# Creating a prompt type input model using only the Transformers pipeline library
from transformers import pipeline

def fill_prompt_template(template, **kwargs):
    return template.format(**kwargs)

# Define your actual prompt here
test_template = """
You are an AI assistant specialized in machine learning and rocketry. Answer the following question:
Question: {question}
Answer:
"""

# defining the base prompt template with variable intro
base_template = """
{intro}
Answer the following question:
Question: {question}
Answer:
"""

# Function to generate text using the custom prompt
def generate_with_template(intro, question, max_new_tokens=256):
    # Fill the new template with the intro and specific question
    prompt = fill_prompt_template(base_template, intro=intro, question=question)

    # Tokenize the prompt
    device = get_cuda_device()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the new text 
    generated_ids = model.generate(inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True)

    # Decode the generated text using tokenizer
    decoded =  tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    return decoded[0]

# generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

# Do an example
intro = "You are an AI assistant specialized in machine learning and rocketry."
question = "What starting mass does a rocket need to have to reach the Moons orbit?"
response = generate_with_template(intro, question)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are an AI assistant specialized in machine learning and rocketry.
Answer the following question:
Question: What starting mass does a rocket need to have to reach the Moons orbit?
Answer:
To reach the Moon's orbit, a rocket needs to have a certain amount of mass, which is dependent on various factors such as the mass of the rocket's payload, fuel capacity, and the rocket's thrust. However, it is worth noting that the Moon's orbit is approximately 238,855 miles (384,400 kilometers) away from Earth, and reaching that distance requires a significant amount of energy.
Assuming that the rocket is unloaded and only has the necessary fuel to reach the Moon's orbit, the starting mass of the rocket would depend on the amount and type of fuel used. For example, if the rocket uses solid rocket fuel, its starting mass would be relatively low since the fuel is already in its combusted form. On the other hand, if the rocket uses liquid rocket fuel, its starting mass would be higher since the fue

In [16]:
text_four = "What about 6 * 5 + 5, what is the asnwer?"
# mistral_llm.invoke(text_four)
print(get_llm(text_four))

What about 6 * 5 + 5, what is the asnwer?
A: 30


In [17]:
qfive = "What is the result of '5 plus 6 times 5'? And think about this, because the problem is two step one; 6 * 5 and then add another 5. What is the answer"
mistral_llm.invoke(qfive)

"What is the result of '5 plus 6 times 5'? And think about this, because the problem is two step one; 6 * 5 and then add another 5. What is the answer to that?\nAnswer: 30"

LangChain - Python framework used for language models


In [18]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
        "Tell me a {adjective} about {content}."
)
prompt.format(adjective="funny", content="chickens")
# adjective in terms of format for prompt. If I wanted to use fact and or scientific fact I'd have to look it up

llm_chain = prompt | mistral_llm
llm_chain.invoke({"adjective": "funny", "content": "chickens"})

"Tell me a funny about chickens.\n\nI'm not much of a joke teller, but I'll give it a try. Here goes: Why don't chickens like to tell secrets? Because they might cluck up!"

#### ConversationBufferMemory class
- Behaves akin to ChatMessageHistory (for precending applications) such that it saves earlier messages.
- Type of memory variable that allows for storing of earlier messages and extracting them in a variable. It is useful for creating chat models that can remember the previous chat instance content and context of the conversation, using earlier message data to better predict and improve subsequent generated answers to new prompts.

1. Custom Applications - Conversation Buffer in LangChain - 

In [19]:
text = "What is Mistral? Write a short answer but apt answer that would pass a college test."
mistral_llm.invoke(text)


'What is Mistral? Write a short answer but apt answer that would pass a college test.\n\nMistral is a type of wind that blows from the Mediterranean Sea towards the French Alps. It is known for its strong and cold winds, which can reach speeds of up to 100 km/h (62 mph). The mistral is a common weather phenomenon in the region and is often associated with extreme conditions, such as heavy snowfall and freezing temperatures.'

- Once again LangChain - the special python framework that supports utilization of various LLM models. A special
- framework specifically designed for large language models.


In [20]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

prompt = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt.format(adjective="funny", content="bears")

llm_chain = prompt | mistral_llm
llm_chain.invoke({"adjective": "funny", "content": "bears"})


"Tell me a funny joke about bears.\n\nWhy don't bears play cards in the wild? Too many cheetahs."

#### How to use Prompts in LangChain
1.  A prompt is a set of instructions or input such as a basic search question that the end user provides to guide the model's response, helping it or setting up the context for the model, and point it in the right direction such that it generates relevant and coherent language-based output. 

In [21]:
llm_chain = prompt | mistral_llm
llm_chain.invoke({"adjective": "funny", "content": "chickens"}, 
                 config={'callbacks': [ConsoleCallbackHandler()]})


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "adjective": "funny",
  "content": "chickens"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "adjective": "funny",
  "content": "chickens"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Tell me a funny joke about chickens."
  ]
}


/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[llm/end] [chain:RunnableSequence > llm:HuggingFacePipeline] [1.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Tell me a funny joke about chickens.\n\nWhy don't chickens like to tell jokes? They might crack each other up and all their eggs will scramble!",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] [1.54s] Exiting Chain run with output:
{
  "output": "Tell me a funny joke about chickens.\n\nWhy don't chickens like to tell jokes? They might crack each other up and all their eggs will scramble!"
}


"Tell me a funny joke about chickens.\n\nWhy don't chickens like to tell jokes? They might crack each other up and all their eggs will scramble!"

In [22]:
# Hmmm...can I use this to pass feedback around to other functions?

In [23]:
from langchain_core.prompts import ChatPromptTemplate
# used to create flexible templated prompts for chat models
chat_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a helpful AI guidance bot. Your name is {name}. Answer the short sentences."),
    ]
)
llm_chain = chat_prompt | mistral_llm
llm_chain.invoke({"name": "Mistral", "user_input": "What is your name?"})
        

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


"System: You are a helpful AI guidance bot. Your name is Mistral. Answer the short sentences.\n\nMistral: Of course, I'm here to assist you with any questions or concerns you may have. How can I help you today?"

In [24]:
# Need the following code if I am to run RetrievalQA for multi chain example
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever


db_docs = ["During a Mars opposition, the distance between Earth and Mars can be as short as 54.6 million kilometers, significantly reducing the travel time and fuel required for a mission."]

# Creating a separate embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={"device": "cuda"},
)

vector_db = FAISS.from_texts(db_docs, embeddings)
retriever = VectorStoreRetriever(vectorstore=vector_db)



/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [25]:
# Prompt Chaining and Snowballing
from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
# from langchain_core.memory import ConversationBufferMemory 
from langchain.memory import ConversationBufferMemory
# from langchain.retrievers import DenseRetriever

# First define the templates for multiple chains
prompt_temp1 = """You are an AI expert in astrophysics. Provide detailed information about the solar system and orbital mechanics.
Question: {question}
Answer:"""

# prompt_temp2 = """Based on the information provided about the solar system, argue for the feasibility of interplanetary travel and colonization.
# Question: {question}
# Answer:"""

prompt_temp2 = """You are an expert aerospace engineer. Using prior information, discuss rocket and spacecraft design for interplanetary flight.
Question: {question}
Answer:"""

prompt_temp3 = """Given the information about interplanetary travel, discuss the potential challenges and solutions for establishing a colony on Mars.
Question: {question}
Answer:"""

# Create PromptTemplate instances from the defined templates
prompt1 = PromptTemplate.from_template(prompt_temp1)
prompt2 = PromptTemplate.from_template(prompt_temp2)
prompt3 = PromptTemplate.from_template(prompt_temp3)


In [26]:
# Step 3: Create the Chains
# You will now create two chains using the defined prompt templates and the language model.


from langchain_core.output_parsers import StrOutputParser

# Initialize the model parser
model_parser = mistral_llm | StrOutputParser()

# Define the chains
chain1 = prompt1 | model_parser
chain2 = prompt2 | model_parser
# Step 4: Combine and Execute the Chains Sequentially
# Combine the chains and run them in sequence using LCEL.

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

# Combine the chains with RunnablePassthrough to maintain context
combined_chain = (
    {"question": RunnablePassthrough(), "history": RunnablePassthrough()}
    | {"chain1_output": chain1, "chain2_output": chain2}
)

# Create a memory buffer for context
memory = ConversationBufferMemory(memory_key="history", input_key="question")

# Final sequential chain with context memory
sequential_chain = (combined_chain | {"final_output": chain2} | prompt2)

# Example usage
response = sequential_chain.invoke({"question": "Tell me about the solar system.", "history": ""})
print(response)

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


text="You are an expert aerospace engineer. Using prior information, discuss rocket and spacecraft design for interplanetary flight.\nQuestion: {'question': 'Tell me about the solar system.', 'history': ''}\nAnswer:"


In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

# Initialize the model parser using the Mistral LLM and StrOutputParser
model_parser = mistral_llm | StrOutputParser()

# Define the first chain, which uses the first prompt template and the model parser
first_chain = (
    {"question": RunnablePassthrough()} | prompt1 | {"question": model_parser}
)

# Define the second chain, which uses the second prompt template and the model parser
chain2 = prompt2 | model_parser

# Define the final chain, which combines the first chain and the second chain, feeding the output of the first chain as input to the second
final_chain = (
    first_chain | {"question": chain2} | prompt3
)

# Example usage of the final_chain with a specific question
response = final_chain.invoke({"question": "Tell me about the solar system."})
# Print the response from the final chain
print(response)

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


text="Given the information about interplanetary travel, discuss the potential challenges and solutions for establishing a colony on Mars.\nQuestion: You are an expert aerospace engineer. Using prior information, discuss rocket and spacecraft design for interplanetary flight.\nQuestion: You are an AI expert in astrophysics. Provide detailed information about the solar system and orbital mechanics.\nQuestion: {'question': 'Tell me about the solar system.'}\nAnswer: The solar system is composed of a star at its center, our Sun, and all the objects that orbit around it, including planets, dwarf planets, moons, asteroids, comets, and other smaller bodies. The Sun makes up almost all of the total mass of the solar system.\nThe solar system is divided into four main regions: the inner planetary region, the asteroid belt, the outer planetary region, and the Oort Cloud. The inner planetary region includes Mercury, Venus, Earth, and Mars, while the outer planetary region includes Jupiter, Satur

In [28]:
new_response = final_chain.invoke({"question": "What is the approximate necessary size of the rocket required in order to make it to Mars?"})
print(new_response)

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


text="Given the information about interplanetary travel, discuss the potential challenges and solutions for establishing a colony on Mars.\nQuestion: You are an expert aerospace engineer. Using prior information, discuss rocket and spacecraft design for interplanetary flight.\nQuestion: You are an AI expert in astrophysics. Provide detailed information about the solar system and orbital mechanics.\nQuestion: {'question': 'What is the approximate necessary size of the rocket required in order to make it to Mars?'}\nAnswer: The size of the rocket required to make it to Mars would depend on several factors such as the mass of the payload, the amount of fuel needed for propulsion, and the design of the rocket itself. However, based on current estimates, a rocket with a diameter of around 10 meters (33 feet) and a length of around 60 meters (200 feet) would be required to carry a payload of around 100 tons to Mars. This is just an estimate and the actual size of the rocket may vary dependin

In [29]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# Define the template for the prompt
template = """You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
{context}
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Chat history: {history}
Question: {question}
Write your answers short. Helpful Answer:"""

# Create the prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["history", "context", "question"]
)

# Initialize the QA chain with memory
qa = RetrievalQA.from_chain_type(
    llm=mistral_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt,
        "memory": ConversationBufferMemory(memory_key="history", input_key="question")
    }
)

# Example usage
context = "Information about the solar system."
history = "Previous chats about planets."
question = "What is the largest planet in the solar system?"
# In calling qa, instaed of "question" use "query"
response = qa({"context": context, "history": history, "query": question})
print(response)

/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/nomad/miniconda3/envs/mistral_7B_mlm/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'context': 'Information about the solar system.', 'history': 'Previous chats about planets.', 'query': 'What is the largest planet in the solar system?', 'result': "You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.\nDuring a Mars opposition, the distance between Earth and Mars can be as short as 54.6 million kilometers, significantly reducing the travel time and fuel required for a mission.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nChat history: \nQuestion: What is the largest planet in the solar system?\nWrite your answers short. Helpful Answer: Jupiter"}


In [30]:
# Setup the individual prompts using promptTemplate
prompt_ONE = PromptTemplate(template=pr_template1, input_variables=["history","context","question"])
# prompt2 = PromptTemplate(template=pr_template2, input_variables=["question", "context"])
# prompt3 = PromptTemplate(template=pr_template3, input_variables=["question", "context"])

# Define a blank retriever
# retriever = SimpleRetriever() # Use the basic retriever class, as a place holder for now.


# Initialize the chain with ConversationBufferMemory
chain1 = RetrievalQA.from_chain_type(
    llm=mistral_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt_ONE,
        "memory": ConversationBufferMemory(memory_key="history", 
                                           input_key="question")
    }
)

# chain2 = RetrievalQA.from_chain_type(
#     llm=mistral_llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={
#         "verbose": False,
#         "prompt": prompt2,
#         "memory": ConversationBufferMemory(memory_key="history", input_key="question")
#     }
# )

# chain3 = RetrievalQA.from_chain_type(
#     llm=mistral_llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={
#         "verbose": False,
#         "prompt": prompt3,
#         "memory": ConversationBufferMemory(memory_key="history", input_key="question")
#     }
# )

        

NameError: name 'pr_template1' is not defined